In [43]:
# Import libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split as sklearn_tt_split

from surprise import Dataset
from surprise import Reader
from surprise import NormalPredictor
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import train_test_split

## File Details - Note, tinkered and abandoned

Start tinkering with Surprise SVD, and with joining on the features and fitting with that. 

So this file doesn't work, we can abandon this one. After research, that's not how Surprise works, surprise is Collaborative Filtering, using only userid and rating.



# Assignment 3 -- Recommendation Systems

* The final challenge is much like Assignment 2 -- but scoped appropriately for the time and your current abilities.
* It is ratings prediction, just like the movielens recommendations we have seen and many other similar problems.
* The features created are based on Beer Reviews from experts on a website.
* Each beer has been scored between 0 and 5 (on a real scale, so 2.75 or 3.5 is OK).
* The official measure is Mean Average Error (MAE) which is pretty intuitive to work with. Everything supports is and it is easy to interpret.
* A set of features have been created based on the reviewer, the written review, and information about the Beer being reviewed.
* Not all features have to be used, and you can easily create new features using the data if you like.
* The features included are:

![title](Images/A3Features.png)

* Sizes of the files are:
|Size | File|
|---|---|
| 1.9G | features.tsv |
| 88B  | header-features.tsv|
| 48B  | header.tsv |
| 15M  | test.tsv |
| 50M  | train.tsv |
| 16M |  val.tsv |


In [44]:
baseDataDir = "C:/Development/Data/COSC2670/Assignment3/A3data/"

In [45]:
# RowID  BeerID  ReviewerID  BeerName  BeerType  Label
df_train = pd.read_csv(baseDataDir + 'train_200k.tsv',sep='\t',
                         names=['RowID','BeerID','ReviewerID',
                                  'BeerName','BeerType','rating'])
df_train.head(10)


,RowID,BeerID,ReviewerID,BeerName,BeerType,rating
0,19,12300,10635,Rauch �r Bock,Rauchbier,4.0
1,21,12300,6547,Rauch �r Bock,Rauchbier,4.5
2,23,12300,9789,Rauch �r Bock,Rauchbier,4.5
3,24,12300,7372,Rauch �r Bock,Rauchbier,5.0
4,25,12300,1302,Rauch �r Bock,Rauchbier,4.5
5,26,12300,704,Rauch �r Bock,Rauchbier,4.5
6,29,12300,1747,Rauch �r Bock,Rauchbier,5.0
7,31,12300,9368,Rauch �r Bock,Rauchbier,4.5
8,32,12300,2568,Rauch �r Bock,Rauchbier,4.0
9,33,12300,6838,Rauch �r Bock,Rauchbier,4.0


In [46]:
# RowID BrewerID ABV DayofWeek Month DayofMonth Year TimeOfDay Gender Birthday Text Lemmatized POS_Tag
df_features = pd.read_csv(baseDataDir + 'features_200k.tsv',sep='\t', names=['RowID','BrewerID','ABV','DayofWeek','Month',
                                                                 'DayofMonth','Year','TimeOfDay','Gender',
                                                                 'Birthday','Text','Lemmatized','POS_Tag'])

df_features.head(10)

,RowID,BrewerID,ABV,DayofWeek,Month,DayofMonth,Year,TimeOfDay,Gender,Birthday,Text,Lemmatized,POS_Tag
0,18,1075,7.4,Mon,Jan,2,2012,15:20:04,Male,unknown,Pours a murky light brown with a 1 inch fizzy ...,pour a murky light brown with a 1 inch fizzy t...,VBZ DT JJ NN JJ IN DT CD NN JJ NN NN WDT VBZ I...
1,19,1075,7.4,Sun,Jan,1,2012,06:46:52,Male,unknown,Faint sudsy head with some with some dissipati...,faint sudsy head with some with some dissipate...,NN JJ NN IN DT IN DT VBG JJ NN . JJ JJ NN . DT...
2,20,1075,7.4,Tue,Nov,29,2011,05:51:44,Male,unknown,A new arrival to the West TN area ... Pours qu...,a new arrival to the West TN area ... pour qui...,"DT JJ NN IN DT NNP NNP NN , VBZ PDT DT NN JJR ..."
3,21,1075,7.4,Sat,Nov,5,2011,22:59:57,Male,unknown,Sampled 10/30/11 - Transferring the notes . A ...,sample 10/30/11 - transfer the note . a ruby p...,VBN CD HYPH VBG DT NNS . DT NN VBP IN DT NN NN...
4,22,1075,7.4,Tue,Nov,1,2011,20:40:21,Male,"Oct 14, 1983",This is my first rauchbier . Pours a burnt amb...,this be my first rauchbier . pour a burn amber...,DT VBZ PRP$ JJ NN . VBZ DT VBN NN . JJ NN . NN...
5,23,1075,7.4,Sat,Oct,29,2011,11:18:34,unknown,unknown,A,a,DT
6,24,1075,7.4,Mon,Oct,17,2011,09:04:02,unknown,unknown,"Pours a mahogany color , rich , with a tan hea...","pour a mahogany color , rich , with a tan head...","VBZ DT NN NN , JJ , IN DT JJ NN . DT NN , VBD ..."
7,25,1075,7.4,Tue,Oct,11,2011,10:41:41,unknown,unknown,Pours light caramel brown with reddish highlig...,pour light caramel brown with reddish highligh...,NNS JJ NN JJ IN JJ NNS . DT JJ JJ NN VBZ RB VB...
8,26,1075,7.4,Tue,Oct,11,2011,10:31:22,Male,"Nov 24, 1974",Poured a slightly cloudy deep amber/red color ...,pour a slightly cloudy deep amber/red color wi...,VBD DT RB JJ JJ VBN NN IN DT JJ NN NN IN NN . ...
9,27,1075,7.4,Mon,Sep,19,2011,14:18:29,Male,"Oct 10, 1988",Big thanks to N2168 for knocking this off my w...,big thanks to n2168 for knock this off my want...,JJ NNS IN NN IN VBG DT RP PRP$ NNS . VBN IN DT...


Okay, Join the dataset to start working on some other features

In [47]:
dfFullData = df_train.join(df_features[["RowID", "BrewerID"]]
    , on="RowID", how="inner", rsuffix="Feat")

dfFullData.head()

,RowID,BeerID,ReviewerID,BeerName,BeerType,rating,RowIDFeat,BrewerID
0,19,12300,10635,Rauch �r Bock,Rauchbier,4.0,37,1075
1,21,12300,6547,Rauch �r Bock,Rauchbier,4.5,40,1075
2,23,12300,9789,Rauch �r Bock,Rauchbier,4.5,42,1075
3,24,12300,7372,Rauch �r Bock,Rauchbier,5.0,43,1075
4,25,12300,1302,Rauch �r Bock,Rauchbier,4.5,45,1075


In [48]:

# Setup the data to be just the Reviewer and the Beer(Item) and the Rating Label we want to learn.
# dfTrainFeatures = df_train.drop(['RowID','BeerName','BeerType'],axis=1)
dfTrainFeatures = dfFullData.drop(['RowID','BeerName','RowIDFeat','BeerType'],axis=1)
dfTrainFeatures = dfTrainFeatures[['BeerID','ReviewerID','BrewerID','rating']]
# dfValiFeatures = df_vali.drop(['RowID','BeerName','BeerType'],axis=1)

dfTrainFeatures.head()

,BeerID,ReviewerID,BrewerID,rating
0,12300,10635,1075,4.0
1,12300,6547,1075,4.5
2,12300,9789,1075,4.5
3,12300,7372,1075,5.0
4,12300,1302,1075,4.5


In [49]:

# The "Random" Prediction Algorithm from Surprise. See the API for a complete description.
# The Algorithm predicts a random rating based on the distribution of the training set, 
# which is assumed to be normal.
algorithm_random = NormalPredictor()

reader = Reader(rating_scale=(0, 5))

In [50]:
# # From the Tute, standard test train split then fit (without CV)
# train_split,vali_split = sklearn_tt_split(dfTrainFeatures[['ReviewerID','BeerID','rating']], test_size=0.2)

# dsTrainFeatures = Dataset.load_from_df(train_split,reader)
# dsValiFeatures = Dataset.load_from_df(vali_split,reader)

# trainsetTrainFeatures = dsTrainFeatures.build_full_trainset()
# NA,valsetTrainFeatures = train_test_split(dsValiFeatures, test_size=1.0)

# print(type(dsTrainFeatures))
# print(type(trainsetTrainFeatures))

# model = algorithm_random.fit(trainsetTrainFeatures)
# predictions = algorithm_random.test(valsetTrainFeatures)

# # Score our predictions with MAE
# # It is around 0.77, which means the a random guess based on the distribution of the data
# # is on average within 0.77 (plus or minus) the true label.
# # Not bad! You can beat it though, I'm sure :).
# # Smaller MAE is the better. Good luck!
# accuracy.mae(predictions,verbose=True)


In [51]:
dsetTrainFeatures = Dataset.load_from_df(dfTrainFeatures,reader)

ValueError: too many values to unpack (expected 3)

In [ ]:
print(type(dsetTrainFeatures.raw_ratings))
print(str(len((dsetTrainFeatures.raw_ratings))))
print(dsetTrainFeatures.raw_ratings[0:50])

<class 'list'>
120564
[(12300, 10635, 4.0, None), (12300, 6547, 4.5, None), (12300, 9789, 4.5, None), (12300, 7372, 5.0, None), (12300, 1302, 4.5, None), (12300, 704, 4.5, None), (12300, 1747, 5.0, None), (12300, 9368, 4.5, None), (12300, 2568, 4.0, None), (12300, 6838, 4.0, None), (12300, 850, 3.0, None), (12300, 9705, 4.0, None), (12300, 3264, 3.5, None), (12300, 2962, 4.5, None), (12300, 2748, 5.0, None), (12300, 757, 3.5, None), (12300, 7207, 4.0, None), (12300, 2849, 4.5, None), (12300, 4737, 4.5, None), (12300, 7826, 4.0, None), (12300, 8720, 4.0, None), (12300, 10680, 4.0, None), (12300, 4094, 4.0, None), (6699, 7162, 4.0, None), (6699, 10135, 2.5, None), (6699, 2465, 4.0, None), (6699, 592, 4.5, None), (6699, 447, 3.5, None), (6699, 4527, 4.0, None), (6699, 10650, 4.0, None), (6699, 10305, 4.0, None), (6699, 837, 3.5, None), (6699, 1082, 4.0, None), (6699, 3158, 4.0, None), (6699, 6718, 3.5, None), (6699, 2849, 4.5, None), (6699, 6872, 5.0, None), (6699, 6330, 5.0, None), (6699

In [ ]:

cv_results = cross_validate(algorithm_random, dsetTrainFeatures, measures=["MAE"], cv=5, verbose=True)

print("Average MAE across 5x KFolds: " + str(np.mean(cv_results["test_mae"])))
print("analyse_maes.append(" + str(np.mean(cv_results["test_mae"])) + ")")

Evaluating MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7265  0.7252  0.7280  0.7326  0.7250  0.7274  0.0028  
Fit time          0.12    0.14    0.14    0.14    0.14    0.14    0.01    
Test time         0.11    0.22    0.21    0.11    0.22    0.17    0.05    
Average MAE across 5x KFolds: 0.727447281995445
analyse_maes.append(0.727447281995445)


In [ ]:

print("Run 1 - Normal with CV")
# Log of Results
analyse_maes = []

analyse_maes.append(0.7290257148745465)
analyse_maes.append(0.7285347965810015)

print("Average MAE over all tests: " + str(np.mean(analyse_maes)))

Run 1 - Normal with CV
Average MAE over all tests: 0.728780255727774


In [ ]:
algorithm_svd = SVD()


svd_results = cross_validate(algorithm_svd, dsetTrainFeatures, measures=["MAE"], cv=5, verbose=True)

print("Average MAE across 5x KFolds: " + str(np.mean(svd_results["test_mae"])))
print("analyse_maes.append(" + str(np.mean(svd_results["test_mae"])) + ")")


Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.4398  0.4381  0.4385  0.4374  0.4405  0.4389  0.0011  
Fit time          3.91    3.99    3.95    3.90    3.93    3.94    0.03    
Test time         0.13    0.13    0.22    0.12    0.13    0.14    0.04    
Average MAE across 5x KFolds: 0.4388707440147794
analyse_maes.append(0.4388707440147794)


In [ ]:

print("Run 2 - SVD")
# Log of Results
analyse_maes = []


print("Average MAE over all tests: " + str(np.mean(analyse_maes)))

Run 2 - SVD
Average MAE over all tests: nan


C:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
